In [1]:
import torch
import datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)

In [ ]:
base_model = "meta-llama/Llama-3.1-8B-Instruct"
dataset_path = "hinglish_journal_dataset2_10k_shuffled.jsonl"  # adjust path if needed

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,   # ✅ use bf16 on L40S
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
model = prepare_model_for_kbit_training(model)

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 9,175,040 || all params: 3,221,924,864 || trainable%: 0.2848


In [5]:
ds = datasets.load_dataset("json", data_files=dataset_path)["train"]

# Format into Alpaca-style instruction data
def format_ex(ex):
    return {
        "text": f"### Journal:\n{ex['journal']}\n\n### Reflection:\n{ex['reflection']}{tokenizer.eos_token}"
    }

ds = ds.map(format_ex)

Map:   0%|          | 0/6446 [00:00<?, ? examples/s]

In [6]:
def tokenize(batch):
    enc = tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc

tok_ds = ds.map(tokenize, batched=True, remove_columns=ds.column_names)

Map:   0%|          | 0/6446 [00:00<?, ? examples/s]

In [7]:
args = TrainingArguments(
    output_dir="/teamspace/studios/this_studio/lora_adapter",
    num_train_epochs=2,                  # can increase to 3 later
    per_device_train_batch_size=4,       # ✅ fits on 48 GB GPU
    gradient_accumulation_steps=8,       # effective batch size = 32
    learning_rate=2e-4,
    bf16=True,
    save_strategy="epoch",
    logging_steps=20,
    optim="paged_adamw_32bit"
)


In [9]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

# Save LoRA adapter
adapter_path = "/teamspace/studios/this_studio/lora_adapter"
model.save_pretrained(adapter_path)

Step,Training Loss
20,1.576600
40,1.431900
60,1.357200
80,1.327500
100,1.288100
120,1.251000
140,1.237200
160,1.211300
180,1.184100
200,1.183900


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


In [10]:
# Interface for fine-tuned model with motivation, mixed sentiments, tips & resources
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import re

# ---------------------------
# Load model and tokenizer
# ---------------------------
def load_model():
    base_model_name = "meta-llama/Llama-3.2-3B-Instruct"
    adapter_path = "/teamspace/studios/this_studio/lora_adapter"
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    
    model = PeftModel.from_pretrained(base_model, adapter_path)
    return model, tokenizer

# ---------------------------
# Clean text
# ---------------------------
def clean_output(text):
    # Remove unwanted markers and notes
    text = re.sub(r"(###|Note:).*", "", text, flags=re.DOTALL).strip()
    return text

# ---------------------------
# Generate reflection
# ---------------------------
def generate_reflection(model, tokenizer, journal_text):
    prompt = (
        "You are a compassionate mental health reflection assistant.\n"
        "For the given journal entry, generate output in 4 clear sections:\n\n"
        "1. Motivation → empathetic and encouraging (can be positive OR mixed depending on the journal tone).\n"
        "2. Improvement Tips → 2–4 practical, culturally relevant suggestions (study, career, fitness, relationships, mental health).\n"
        "3. Guided Resources → 2–3 helpful links (YouTube talks, meditation apps, motivational articles, podcasts).\n"
        "4. Closing Note → warm, human-like closing lines that leave hope & reassurance.\n\n"
        "Keep the response realistic, emotionally nuanced, and not repetitive.\n\n"
        f"Journal Entry:\n{journal_text}\n\nResponse:"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=700,   # allow long responses
            temperature=0.85,
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text.split("Response:")[-1].strip()
    return clean_output(response)

# ---------------------------
# Main
# ---------------------------
if __name__ == "__main__":
    print("Loading model...")
    model, tokenizer = load_model()
    print("✅ Model loaded!")

    # Example journals
    example_entries = [
        "Aaj ka din accha tha, lekin thoda guilt feel hua kyunki gym skip kar diya.",
        "Meri best friend se fight ho gayi, ab lag raha hai shayad galti meri thi.",
        "Workload itna zyada hai ki anxiety ho rahi hai, bas chhutti chahiye.",
        "Aaj ghar pe family ke saath maza aaya, lekin thoda stress bhi tha kal ke exam ka.",
        "Kal raat neend nahi aayi, mann bahut heavy tha aur khud pe doubt hua."
    ]

    print("\n🔹 Testing with mixed sentiment journal entries:\n")
    for i, entry in enumerate(example_entries, 1):
        print(f"--- Example {i} ---")
        print(f"Journal: {entry}")
        reflection = generate_reflection(model, tokenizer, entry)
        print(f"{reflection}\n")
        print("="*100 + "\n")


Loading model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded!

🔹 Testing with mixed sentiment journal entries:

--- Example 1 ---
Journal: Aaj ka din accha tha, lekin thoda guilt feel hua kyunki gym skip kar diya.
**Motivation**
It's amazing you took time to acknowledge your feelings; acknowledging our emotions is really brave! Keep moving forward, even small steps like going to the gym are worth it for your well-being.

**Improvement Tips**
Here are some tips that might help you connect with your inner motivation: try writing down three things you enjoyed doing today or did well for yourself. It can also be helpful to set smaller goals so your daily routine doesn't feel overwhelming.

**Guided Resources**
Did you know there's an app called Fit


--- Example 2 ---
Journal: Meri best friend se fight ho gayi, ab lag raha hai shayad galti meri thi.
**Motivation**
Kahi baar chhoti-choti zindagi ke issues le kar sab kuch sambhalna mushkil hota hai. Aapki feelings bilkul valid hain, aur it's okay to express them; remember, you're strong